## Setup

In [1]:
import relationalai as rai
from relationalai.graphs import Graph

In [2]:
model = rai.Model("SocialMoneyNetwork")
Account = model.Type("Account")
Transaction = model.Type("Transaction")

## Data Loading

In [3]:
with model.rule():
    account_id, account_type, name = rai.csv.load_file(model, "accounts.csv")
    a = Account.add(id=account_id)
    with account_type:
        a.set(account_type=account_type)
    with name:
        a.set(name=name)

with model.rule():
    from_account, to_account, amount, type, date = rai.csv.load_file(model, "transactions.csv")
    t = Transaction.add(
        from_account=Account(id=from_account),
        to_account=Account(id=to_account),
        date=date
    )
    with amount:
        t.set(amount=amount)
    with type:
        t.set(type=type)

## Inspecting tables

In [5]:
with model.query() as select:
    a = Account()
    response = select(a.id, a.account_type, a.name)

response.results

,id,account_type,name
0,merchant_1,Merchant,Blossom Florist & Gifts
1,merchant_10,Merchant,Blue Ribbon Butchers
2,merchant_2,Merchant,The Rustic Loaf Bakery
3,merchant_3,Merchant,Bella's Café & Espresso
4,merchant_4,Merchant,Sundown Brewery
...,...,...,...
105,user_95,User,John Whitehead
106,user_96,User,Anna Henderson
107,user_97,User,Aaron Wise
108,user_98,User,Deborah Figueroa


In [6]:
with model.query() as select:
    t = Transaction()
    response = select(t.from_account.id, t.to_account.id, t.amount, t.type, t.date)

response.results

,id,id2,amount,type,date
0,merchant_1,user_10,7.68,refund,2024-01-25
1,merchant_1,user_6,93.18,refund,2023-12-10
2,merchant_1,user_94,124.10,refund,2023-12-21
3,merchant_1,user_98,61.08,refund,2023-11-13
4,merchant_10,user_14,9.28,refund,2023-11-28
...,...,...,...,...,...
296,user_98,user_21,15.79,transaction,2024-01-14
297,user_98,user_44,15.77,transaction,2024-01-27
298,user_99,merchant_4,135.36,transaction,2024-02-03
299,user_99,merchant_6,96.12,transaction,2023-12-24


## Graph Algorithms

In [4]:
graph = Graph(model)

In [5]:
with model.rule():
    a = Account()
    t = Transaction()
    graph.nodes.add(a)
    graph.edges.add(t.from_account, t.to_account)

In [6]:
with model.rule():
    a = Account()
    degree = graph.compute.degree(a)
    graph.nodes.add(a, degree=degree, label=a.name)

In [7]:
with model.rule():
    a = Account()
    pagerank = graph.compute.pagerank(a)
    graph.nodes.add(a, pagerank=pagerank)

In [8]:
graph.fetch()

AttributeError: 'SnowflakeCursor' object has no attribute 'results'

In [ ]:
graph.nodes()

In [ ]:
graph.visualize()